In [1]:
import numpy as np
import os
import pandas as pd
import random
from tqdm import tqdm
import xgboost as xgb
import pickle as pickle
import os.path

import scipy
from sklearn.metrics import fbeta_score

from PIL import Image
print("------Imports complete-----")

------Imports complete-----


Set path variables:

In [46]:
random_seed = 0
random.seed(random_seed)
np.random.seed(random_seed)

train_path = "/media/alex/B44254FE4254C730/Users/Alex/Downloads/Kaggle Data/train-jpg/train-jpg/"
test_path = "/media/alex/B44254FE4254C730/Users/Alex/Downloads/Kaggle Data/test-jpg/"

train = pd.read_csv("/home/alex/Desktop/Rainforest/Data/train_v2.csv")
test =  pd.read_csv("/home/alex/Desktop/Rainforest/Data/sample_submission_v2.csv")
#Location for saving serialized objects
obj_save_path = "/home/alex/Desktop/Rainforest/Models/XGB/Objects/"
subm_output_path = "/home/alex/Desktop/Rainforest/Submissions/"

Feature-Extraction Method:

In [3]:
def extract_features(df, data_path):
	#create a copy of original panda dataframe so that extracted features can be added.
	im_features = df.copy()
	r_mean = []
	g_mean = []
	b_mean = []

	r_std=[]
	g_std=[]
	b_std=[]

	r_max=[]
	g_max=[]
	b_max=[]
	
	r_min=[]
	g_min=[]
	b_min=[]

	r_kurtosis=[]
	g_kurtosis=[]
	b_kurtosis=[]

	r_skewness=[]
	g_skewness=[]
	b_skewness=[]

	for image_name in tqdm(im_features.image_name.values, miniters=1000):
		
		if "file_" in image_name:
			continue 
		im = Image.open(data_path + image_name + '.jpg')
		#creating 256x256x3 array
		im= np.array(im)[:,:,:3]
		
		# im[:,:,0] has dimension 256x256; .ravel() flattens into 256^2 x 1
		r = im[:,:,0].ravel()
		g = im[:,:,1].ravel()
		b = im[:,:,2].ravel()
		
		r_mean.append(np.mean(r))
		g_mean.append(np.mean(g))
		b_mean.append(np.mean(b))

		r_std.append(np.std(r))
		g_std.append(np.std(g))
		b_std.append(np.std(b))

		r_max.append(np.max(r))
		g_max.append(np.max(g))
		b_max.append(np.max(b))
		
		r_min.append(np.min(r))
		g_min.append(np.min(g))
		b_min.append(np.min(b))
		
		r_kurtosis.append(scipy.stats.kurtosis(r))
		g_kurtosis.append(scipy.stats.kurtosis(g))
		b_kurtosis.append(scipy.stats.kurtosis(b))
		
		r_skewness.append(scipy.stats.skew(r))
		g_skewness.append(scipy.stats.skew(g))
		b_skewness.append(scipy.stats.skew(b))

	#Add extracted features to pandas frame
	#print(im_features['r_mean'].shape)
	#print(r_mean.shape)
	im_features['r_mean'] = r_mean
	im_features['g_mean'] = g_mean
	im_features['b_mean'] = b_mean
	
	im_features['r_std'] = r_std
	im_features['g_std'] = g_std
	im_features['b_std'] = b_std
	
	im_features['r_max'] = r_max
	im_features['g_max'] = g_max
	im_features['b_max'] = b_max
	
	im_features['r_min'] = r_min
	im_features['g_min'] = g_min
	im_features['b_min'] = b_min
	
	im_features['r_kurtosis'] = r_kurtosis
	im_features['g_kurtosis'] = g_kurtosis
	im_features['b_kurtosis'] = b_kurtosis
	
	im_features['r_skewness'] = r_skewness
	im_features['g_skewness'] = g_skewness
	im_features['b_skewness'] = b_skewness
	
	#return a pandas dataframe with above features extracted
	return im_features

In [4]:
#Check for saved objects.  If not found extract features.
if(os.path.isfile(obj_save_path + "train_features.p")):
    #Try loading frame, if fails then re-extract.
    try:
        file = open(obj_save_path + "train_features.p",'rb')
        train_features = pickle.load(file)
        file.close()
        print("---train_features successfully loaded---")
    except:
        print("---loading of train_features failed--")
        train_features = extract_features(train,train_path)
        pickle.dump(train_features, open(obj_save_path + 'train_features.p','wb'))
else:
    print("train_features.p not found-- training model")
    train_features = extract_features(train,train_path)
    pickle.dump(train_features, open(obj_save_path + 'train_features.p','wb'))



---train_features successfully loaded---


In [5]:
#Extract features on test set:
print('Extracting test features')

if(os.path.isfile(obj_save_path + "test_features.p")):
    try:
        file = open(obj_save_path + "test_features.p",'rb')
        test_features = pickle.load(file)
        file.close()
        print("---test_features successfully loaded---")
    except:
        print("---loading of test_features failed--")
        test_features = extract_features(test,test_path)
        pickle.dump(test_features, open(obj_save_path + 'test_features.p','wb'))

else:
    print("test_features.p not found-- extracting features")
    test_features = extract_features(test,test_path)
    pickle.dump(test_features, open(obj_save_path + 'test_features.p','wb'))

Extracting test features
---test_features successfully loaded---


# Data Formatting:

In [6]:
#Drop labels, keep only predictors.  Result is 40479 x 18 matrix
X = np.array(train_features.drop(['image_name','tags'],axis = 1))
y_train = []

#Define a function 'flatten'
#Very clever way to to take a list of lists and convert to a simple list
#e.g. from [[1,2],[3,4]] --> [1,2,3,4] 
flatten = lambda l: [item for sublist in l for item in sublist]

#Go through each observation, gather all tags, get all unique values
tmp = []
for l in train_features['tags'].values:
    tmp.append(l.split(' '))
    
labels = np.array(list(set(flatten(tmp))))


In [7]:
#Create dictionary of labels
label_map = {l: i for i, l in enumerate(labels)}


The following for-loop creates an nx17 binary indicator matrix for the 17 possible labels.

In [8]:
for tags in tqdm(train.tags.values, miniters = 1000):
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1
    y_train.append(targets)

#Convert python list to numpy array.
#np.uint8 is data type: unsigned 8-bit integer
y = np.array(y_train, np.uint8)

100%|██████████| 40479/40479 [00:00<00:00, 196931.14it/s]


In [26]:
#X.shape gives extracted features in observation matrix
print('X.shape = ' + str(X.shape))
#y.shape gives number of possible labels
print('y.shape = ' + str(y.shape))

#set classes to 17
n_classes = y.shape[1]

X_test = np.array(test_features.drop(['image_name','tags'], axis = 1))

#Create a null matrix of prediction count x 17
y_pred = np.zeros((X_test.shape[0], n_classes))


X.shape = (40479, 18)
y.shape = (40479, 17)


# Train All 17 Models!

In [27]:
for class_i in tqdm( range(n_classes), miniters = 1 ):
    model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=100, \
                              silent=True, objective='binary:logistic', nthread=-1, \
                              gamma=0, min_child_weight=1, max_delta_step=0, \
                              subsample=1, colsample_bytree=1, colsample_bylevel=1, \
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, \
                              base_score=0.5, seed=random_seed, missing=None)
    model.fit(X, y[:, class_i])
    #Apply model i
    y_pred[:,class_i] = model.predict_proba(X_test)[:,1]

100%|██████████| 17/17 [00:38<00:00,  2.77s/it]


In [30]:
preds = [' '.join(labels[y_pred_row > 0.21]) for y_pred_row in y_pred]

In [37]:
#Arbitrary cutoff of .21 imposed-- I have no idea why this number is chosen.
preds = [' '.join(labels[y_pred_row > 0.21]) for y_pred_row in y_pred]



In [47]:
#Build submission file
subm = pd.DataFrame()
subm['image_name'] = test_features.image_name.values
subm['tags'] = preds
subm.to_csv(str(subm_output_path + 'submission.csv'), index = False)